In [2]:
import pandas as pd
data = pd.read_csv('/content/part2_datamerged.csv')
data.head()

,latitude,longitude,sand % topsoil,sand % subsoil,silt % topsoil,silt% subsoil,clay % topsoil,clay % subsoil,pH water topsoil,pH water subsoil,...,SnowfSummer,SnowfWinter,TairFall,TairSpring,TairSummer,TairWinter,WindFall,WindSpring,WindSummer,WindWinter
0,22.25,6.25,24.40,24.95,13.275,13.80,12.375,11.225,3.50,3.575,...,0.0,0.0,295.799433,301.152019,304.940717,289.933618,3.477711,4.141446,3.808844,4.091673
1,22.25,6.75,58.90,56.00,16.200,17.00,24.900,27.000,7.10,7.200,...,0.0,0.0,297.920661,303.343371,305.586701,292.493344,3.677101,4.073748,3.960226,4.813505
2,33.25,6.25,45.08,42.25,20.950,23.65,21.690,24.450,7.59,7.700,...,0.0,0.0,293.622162,298.524452,307.309000,286.554831,3.905654,4.786524,4.385125,4.192446
3,25.25,2.25,45.08,42.25,29.010,30.68,25.980,26.810,7.59,7.730,...,0.0,0.0,296.827460,302.241684,308.812473,290.330801,4.203175,4.691022,4.591938,4.683384
4,25.25,2.75,45.08,42.25,29.010,30.68,25.980,26.810,7.59,7.730,...,0.0,0.0,296.827460,302.241684,308.812473,290.330801,4.203175,4.691022,4.591938,4.683384


In [3]:
print(data.columns)

Index(['latitude', 'longitude', 'sand % topsoil', 'sand % subsoil',
       'silt % topsoil', 'silt% subsoil', 'clay % topsoil', 'clay % subsoil',
       'pH water topsoil', 'pH water subsoil', 'OC % topsoil', 'OC % subsoil',
       'N % topsoil', 'N % subsoil', 'BS % topsoil', 'BS % subsoil',
       'CEC topsoil', 'CEC subsoil', 'CEC clay topsoil', 'CEC Clay subsoil',
       'CaCO3 % topsoil', 'CaCO3 % subsoil', 'BD topsoil', 'BD subsoil',
       'C/N topsoil', 'C/N subsoil', 'geometry', 'PSurfFall', 'PSurfSpring',
       'PSurfSummer', 'PSurfWinter', 'QairFall', 'QairSpring', 'QairSummer',
       'QairWinter', 'RainfFall', 'RainfSpring', 'RainfSummer', 'RainfWinter',
       'SnowfFall', 'SnowfSpring', 'SnowfSummer', 'SnowfWinter', 'TairFall',
       'TairSpring', 'TairSummer', 'TairWinter', 'WindFall', 'WindSpring',
       'WindSummer', 'WindWinter'],
      dtype='object')


In [5]:
# List of seasonal columns
seasons = {
    "Fall": ['PSurfFall', 'QairFall', 'RainfFall', 'SnowfFall', 'TairFall', 'WindFall'],
    "Spring": ['PSurfSpring', 'QairSpring', 'RainfSpring', 'SnowfSpring', 'TairSpring', 'WindSpring'],
    "Summer": ['PSurfSummer', 'QairSummer', 'RainfSummer', 'SnowfSummer', 'TairSummer', 'WindSummer'],
    "Winter": ['PSurfWinter', 'QairWinter', 'RainfWinter', 'SnowfWinter', 'TairWinter', 'WindWinter']
}
# Common columns (all columns excluding seasonal ones)
seasonal_columns = [col for season in seasons.values() for col in season]
common_columns = [col for col in data.columns if col not in seasonal_columns]

# Create separate files for each season
for season, cols in seasons.items():
    seasonal_df = data[common_columns + cols]
    output_filename = f"{season}_data.csv"
    seasonal_df.to_csv(output_filename, index=False)
    print(f"Created {output_filename}")

Created Fall_data.csv
Created Spring_data.csv
Created Summer_data.csv
Created Winter_data.csv


In [10]:
import pandas as pd
import numpy as np  # Import NumPy for matrix operations
from sklearn.preprocessing import MinMaxScaler

# List of seasonal files
files = ["Fall_data.csv", "Spring_data.csv", "Summer_data.csv", "Winter_data.csv"]

# Correlation threshold for vertical reduction
correlation_threshold = 0.9

# Process each file
for file in files:
    # Load the data
    df = pd.read_csv(file)

    # Remove the 'geometry' column if it exists
    if 'geometry' in df.columns:
        df = df.drop(columns=['geometry'])

    # Step 5.1: Vertical reduction (Remove highly correlated features)
    corr_matrix = df.corr().abs()
    upper_triangle = corr_matrix.where(
        ~np.tril(np.ones(corr_matrix.shape)).astype(bool)
    )  # Get the upper triangle of the correlation matrix
    high_corr_features = [
        column for column in upper_triangle.columns if any(upper_triangle[column] > correlation_threshold)
    ]
    df_reduced = df.drop(columns=high_corr_features)

    # Step 5.2: Horizontal reduction (Remove duplicate rows)
    df_reduced = df_reduced.drop_duplicates()

    # Step 6: Normalization (Min-Max Scaling)
    scaler = MinMaxScaler()
    df_normalized = pd.DataFrame(scaler.fit_transform(df_reduced), columns=df_reduced.columns)

    # Save the preprocessed data
    output_file = file.replace(".csv", "_processed.csv")
    df_normalized.to_csv(output_file, index=False)
    print(f"Preprocessed file saved as {output_file}")


Preprocessed file saved as Fall_data_processed.csv
Preprocessed file saved as Spring_data_processed.csv
Preprocessed file saved as Summer_data_processed.csv
Preprocessed file saved as Winter_data_processed.csv


In [11]:
data = pd.read_csv('/content/Winter_data_processed.csv')
data.head()

,latitude,longitude,sand % topsoil,silt % topsoil,clay % topsoil,pH water topsoil,OC % topsoil,OC % subsoil,N % topsoil,N % subsoil,CEC topsoil,CaCO3 % topsoil,C/N topsoil,C/N subsoil,PSurfWinter,QairWinter,RainfWinter,SnowfWinter,TairWinter,WindWinter
0,0.171429,0.725,0.099031,0.201471,0.191886,0.097202,0.139193,0.390750,0.196429,0.296455,0.131054,0.029815,0.086705,0.042969,0.350259,0.157214,0.0,0.0,0.708311,0.663585
1,0.171429,0.750,0.533157,0.258824,0.403171,0.627393,0.368260,0.454545,0.428571,0.108485,0.226184,0.002982,0.433526,0.375000,0.492980,0.206328,0.0,0.0,0.873486,0.892080
2,0.800000,0.725,0.359255,0.351961,0.349022,0.699558,0.159141,0.326954,0.250000,0.296455,0.218201,0.274299,0.306358,0.500000,0.798413,0.521601,0.0,0.0,0.490282,0.695485
3,0.342857,0.525,0.359255,0.510000,0.421390,0.699558,0.159141,0.591707,0.335714,0.752954,0.267429,0.327370,0.306358,0.500000,0.706555,0.283907,0.0,0.0,0.733940,0.850890
4,0.342857,0.550,0.359255,0.510000,0.421390,0.699558,0.159141,0.591707,0.335714,0.752954,0.267429,0.327370,0.306358,0.500000,0.706555,0.283907,0.0,0.0,0.733940,0.850890


In [ ]:
from sklearn.feature_selection import VarianceThreshold

# Define a low variance threshold
variance_threshold = 0.01

# Process each preprocessed file
processed_files = ["Fall_data_processed.csv", "Spring_data_processed.csv", "Summer_data_processed.csv", "Winter_data_processed.csv"]

for file in processed_files:
    # Load the normalized data
    df = pd.read_csv(file)

    # Step 7: Eliminate low variance features
    selector = VarianceThreshold(threshold=variance_threshold)
    reduced_data = selector.fit_transform(df)

    # Get the remaining feature names
    selected_features = df.columns[selector.get_support()]
    removed_features = df.columns[~selector.get_support()]

    # Print the removed features
    print(f"Features eliminated in {file}:")
    print(removed_features.tolist())

    # Create a DataFrame with the reduced features
    df_reduced = pd.DataFrame(reduced_data, columns=selected_features)

    # Save the final reduced data
    output_file = file.replace("_processed.csv", "_final.csv")
    df_reduced.to_csv(output_file, index=False)
    print(f"Final reduced file saved as {output_file}\n")


In [12]:
from sklearn.feature_selection import VarianceThreshold
import pandas as pd

# Define a low variance threshold
variance_threshold = 0.01

# List of preprocessed files
processed_files = ["Fall_data_processed.csv", "Spring_data_processed.csv", "Summer_data_processed.csv", "Winter_data_processed.csv"]

for file in processed_files:
    # Load the normalized data
    df = pd.read_csv(file)

    # Step 7: Eliminate low variance features
    selector = VarianceThreshold(threshold=variance_threshold)
    reduced_data = selector.fit_transform(df)

    # Get the remaining and removed feature names
    selected_features = df.columns[selector.get_support()]
    removed_features = df.columns[~selector.get_support()]

    # Print the removed features
    print(f"Features eliminated in {file}:")
    print(removed_features.tolist())

    # Create a DataFrame with the reduced features
    df_reduced = pd.DataFrame(reduced_data, columns=selected_features)

    # Save the final reduced data
    output_file = file.replace("_processed.csv", "_final.csv")
    df_reduced.to_csv(output_file, index=False)
    print(f"Final reduced file saved as {output_file}\n")


Features eliminated in Fall_data_processed.csv:
['clay % topsoil', 'OC % topsoil', 'CEC topsoil', 'RainfFall', 'SnowfFall']
Final reduced file saved as Fall_data_final.csv

Features eliminated in Spring_data_processed.csv:
['clay % topsoil', 'OC % topsoil', 'CEC topsoil', 'RainfSpring', 'SnowfSpring']
Final reduced file saved as Spring_data_final.csv

Features eliminated in Summer_data_processed.csv:
['clay % topsoil', 'OC % topsoil', 'CEC topsoil', 'RainfSummer', 'SnowfSummer']
Final reduced file saved as Summer_data_final.csv

Features eliminated in Winter_data_processed.csv:
['clay % topsoil', 'OC % topsoil', 'CEC topsoil', 'RainfWinter', 'SnowfWinter']
Final reduced file saved as Winter_data_final.csv

